In [18]:
## IMPORTS
import pandas as pd
import numpy as np
from datetime import date,datetime, timedelta
import investpy
import time

In [19]:
SectorDict = {
    "S&P 500 Energy":"XLE",
    "S&P 500 Information Technology":"XLK",
    "S&P 500 Telecom Services":"XLC",
    "S&P 500 Consumer Staples":"XLP",
    "S&P 500 Health Care":"XLV",
    "S&P 500 Consumer Discretionary":"XLY",
    "S&P 500 Industrials":"XLI",
    "S&P 500 Utilities":"XLU",
    "S&P 500 Materials":"XLB",
    "S&P 500 Financials":"XLF",
    "S&P 500 Real Estate":"XLRE"}
now = datetime.now()
sectorDfList = []
for sector in SectorDict.keys():
    df = investpy.indices.get_index_historical_data(sector, 'United States', '01/01/2010', '01/01/2022', as_json=False, order='ascending', interval='Daily')
    sectorDfList.append(df)
    sectorDfList[-1].to_pickle("stockData/sectorCharts/"+SectorDict[sector]+".pkl")
    time.sleep(5)

In [20]:
SectorList = ["XLE", "XLK", "XLC", "XLP", "XLV", "XLY", "XLI", "XLU", "XLB", "XLF", "XLRE"]

In [21]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
spDf = investpy.indices.get_index_historical_data("S&P 500", 'United States', '01/01/2002', '01/01/2022', as_json=False, order='ascending', interval='Daily')
def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iloc[index]['Close']/df.iloc[index-30]['Close']
def calculateSlope(df,index):
    if index < 31:
        return 0
    else:
        return df.iloc[index]['MA30']/df.iloc[index-1]['MA30']
i = 0
for df in sectorDfList:
    df['MA30'] = df['Close'].rolling(window=30).apply(lambda x: np.sum(x)/30)
    df['MA30Slope'] = df.apply(lambda x: calculateSlope(df,df.index.get_loc(x.name)),axis=1)
    df['Volume'] = df['Volume'].replace(to_replace=0, method='ffill')
    df['percent'] = df.apply(lambda x: product(df,df.index.get_loc(x.name)),axis=1)
    spDf['percent'] = spDf.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
    df['RS'] = df['percent'] - spDf['percent']
    df.drop(['Open','High','Low','Close','Volume','percent','Currency'],axis=1)
    df.to_pickle("stockData/sectorCharts/"+SectorList[i]+".pkl")
    i += 1

In [22]:
goodSectorDf = sectorDfList[0].drop(['Open','High','Low','Close','Currency','MA30','MA30Slope','RS'],axis=1)

In [23]:
from stage import fullPrint
goodSectorDf.index = sectorDfList[0].index
fullPrint(sectorDfList[1][['MA30Slope','RS']])

            MA30Slope        RS
Date                           
2010-01-04   0.000000 -1.020896
2010-01-05   0.000000 -1.038013
2010-01-06   0.000000 -1.041929
2010-01-07   0.000000 -1.032045
2010-01-08   0.000000 -1.035572
2010-01-11   0.000000 -1.032729
2010-01-12   0.000000 -1.040981
2010-01-13   0.000000 -1.045681
2010-01-14   0.000000 -1.035712
2010-01-15   0.000000 -1.024152
2010-01-19   0.000000 -1.045740
2010-01-20   0.000000 -1.028988
2010-01-21   0.000000 -1.011992
2010-01-22   0.000000 -0.999835
2010-01-25   0.000000 -1.000757
2010-01-26   0.000000 -0.990765
2010-01-27   0.000000 -0.991947
2010-01-28   0.000000 -0.973450
2010-01-29   0.000000 -0.969258
2010-02-01   0.000000 -0.981978
2010-02-02   0.000000 -1.006605
2010-02-03   0.000000 -0.995292
2010-02-04   0.000000 -0.954275
2010-02-05   0.000000 -0.953641
2010-02-08   0.000000 -0.943021
2010-02-09   0.000000 -0.950323
2010-02-10   0.000000 -0.947108
2010-02-11   0.000000 -0.957619
2010-02-12   0.000000 -0.954804
2010-02-

In [ ]:
goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
goodSectorDf['BadSectors'] = [[] for _ in range(len(goodSectorDf))]
for date in sectorDfList[0].index:
    listOfSector = []
    listOfBadSector = []
    for df in sectorDfList:
        try:
            if df.at[date,'MA30Slope'] > 1 or df.loc[date,'RS'] > 0.1:
                if df.at[date,'close'] > df.at[date,'MA30']:
                    listOfSector.append(df.at[date,'ticker'])
            if (df.at[date,'MA30Slope'] < 0.995 and df.loc[date,'RS'] < 0) or df.at[date, 'MA30Slope'] < 0.975:
                if df.at[date,'close'] < df.at[date,'MA30']:
                    listOfBadSector.append(df.at[date,'ticker'])
        except: 
            print(df.iloc[-1]['ticker'])
    goodSectorDf.at[date,'Sectors'] = set(listOfSector)
    goodSectorDf.at[date,'BadSectors'] = set(listOfBadSector)

XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
XLC
XLRE
X

In [ ]:
from stage import fullPrint
fullPrint(goodSectorDf)
goodSectorDf.to_pickle('stockData/goodSector.pkl')

                  open        high         low   adjclose    volume   percent  \
1999-01-08   24.000000   24.562500   23.375000  13.600327    421100  0.000000   
1999-01-15   22.562500   22.750000   22.453125  12.891050     10700  0.000000   
1999-01-22   22.750000   22.812500   22.562500  12.820119     19300  0.000000   
1999-01-29   21.687500   21.812500   21.437500  12.376831     18600  0.000000   
1999-02-05   22.312500   22.937500   22.187500  12.979712     66000  0.000000   
1999-02-12   22.375000   22.625000   22.156250  12.571879    242300  0.000000   
1999-02-19   21.937500   21.968750   21.531250  12.323625    130300  0.000000   
1999-02-26   21.843750   22.000000   21.625000  12.270435     24900  0.000000   
1999-03-05   23.000000   23.031250   22.625000  12.970847     92700  0.000000   
1999-03-12   24.875000   25.062500   24.250000  13.901767    604300  0.000000   
1999-03-19   25.250000   25.281250   24.625000  14.111900    260400  0.000000   
1999-03-26   24.687500   24.